In [37]:
import random
from datetime import datetime, timedelta
import json

def random_date(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())))

def generate_high_frequency_transactions(customer_id, start_date, num_transactions):
    transactions = []
    for _ in range(num_transactions):
        transactions.append({
            "transaction_id": f"T{random.randint(10000, 99999)}",
            "date_time": random_date(start_date, start_date + timedelta(days=1)).isoformat(),
            "amount": random.uniform(10, 1000),
            "currency": random.choice(["USD", "EUR", "GBP"]),
            "merchant_details": f"Merchant{random.randint(1, 20)}",
            "customer_id": customer_id,
            "transaction_type": random.choice(["purchase", "withdrawal"]),
            "location": f"City{random.randint(11, 20)}"  # Different from customer's city
        })
    return transactions

def generate_data(num_transactions, num_customers):
    customers = []
    transactions = []
    external_data = {
        "blacklist_info": [f"Merchant{random.randint(21, 30)}" for _ in range(10)],
        "credit_scores": {},
        "fraud_reports": {}
    }

    for i in range(num_customers):
        customer_id = f"C{i:03}"
        customer_city = f"City{random.randint(1, 10)}"
        customers.append({
            "customer_id": customer_id,
            "account_history": [],
            "demographics": {"age": random.randint(18, 70), "location": customer_city},
            "behavioral_patterns": {"avg_transaction_value": random.uniform(50, 500)}
        })
        external_data["credit_scores"][customer_id] = random.randint(300, 850)
        external_data["fraud_reports"][customer_id] = random.randint(0, 5)

    for i in range(num_transactions):
        customer_id = f"C{random.randint(0, num_customers-1):03}"
        transaction = {
            "transaction_id": f"T{i:05}",
            "date_time": random_date(datetime(2020, 1, 1), datetime(2023, 1, 1)).isoformat(),
            "amount": random.uniform(10, 1000) * (10 if random.random() < 0.4 else 1),  # 5% chance of high amount
            "currency": random.choice(["USD", "EUR", "GBP"]),
            "merchant_details": f"Merchant{random.randint(1, 20)}",
            "customer_id": customer_id,
            "transaction_type": random.choice(["purchase", "withdrawal"]),
            "location": f"City{random.randint(1, 10)}"
        }
        transactions.append(transaction)
        for customer in customers:
            if customer['customer_id'] == customer_id:
                customer['account_history'].append(transaction['transaction_id'])
                break

    for customer in random.sample(customers, num_customers // 40):  # 40% of customers
        transactions.extend(generate_high_frequency_transactions(customer['customer_id'], datetime(2022, 1, 1), 10))

    return transactions, customers, external_data

transactions, customers, external_data = generate_data(1000, 100)

customers= json.dumps(customers)
with open("../data/customers.json",mode="w") as file:
    for customer in customers:
         file.write(customer)

transactions= json.dumps(transactions)
with open("../data/transactions.json",mode="w") as file:
    for transaction in transactions:
         file.write(transaction)

with open("../data/external_data.json",mode="w") as file:
# for extern_data in external_data:
        external_data_dic=json.dump(external_data,file)

# print()       


In [16]:
import requests as re
import pandas as pd
import json
import logging
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType, MapType

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s - %(name)s', datefmt='%d-%b-%y %H:%M:%S')
print(re.__version__)

def initial_session():
    spark = SparkSession.builder.appName("Customers").getOrCreate()
    return spark
def get_customers_data():
 
    request=re.get("http://localhost:8000/api/v1/customer")
    response=request.json()
    return response

def main():
    spark=initial_session()
    data=get_customers_data()
    data=json.dumps(data).format("utf-8")
    schema=StructType([
        StructField("customer_id",StringType(),True),
        StructField("account_history",ArrayType(StringType()),True),
        StructField("demographics",MapType(StringType(),StringType()),True),
        StructField("behavioral_patterns",MapType(StringType(),FloatType()),True)
    ])
    
    df=spark.read.schema(schema).json(data).option("multiline","true")
    df.printSchema()
    df.show()
if __name__=="__main__":
 main()        


2.32.3


KeyError: '"customer_id"'

In [21]:
import pandas as pd

request=re.get("http://localhost:8000/api/v1/customer")
response=request.json()
data=json.dumps(response)
df=pd.read_json(data)
df.value_counts()

/tmp/ipykernel_54783/3656885996.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_json(data)


TypeError: unhashable type: 'list'